# Set up SFINCS model

This notebook guides a user through the set up of a SFINCS model for the German Baltic coast in the Barth area. The set up will use locally sourced data as much as possible to aim for the most accurate results.

In [ ]:
from pathlib import Path
import xarray as xr
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

from hydromt_sfincs import SfincsModel
from hydromt_sfincs import utils
from hydromt.log import setuplog


## Basic Config

### Model path & data location

First we define some basic aspects of the model. Here we set data locations i.e. where the model will be saved and how this particular version is called. We also define where to find the necessary data for the setup, though the actual datafiles are defined later when we need them.

In [ ]:
# Top project folder
project_root = Path().resolve()

# Folder containing different model versions
model_root = project_root/"models"

# This model version's name
model_name = "20m_final"

# Data folder
data_root = project_root / "data"

### Basic model parameters

Here we set some necessary parameter, being the grid resolution (in meters), the point on the boundary where the waterlevels are forced, and the points at which timeseries are written to the output. Note that the units (meters) and coordinates are those of the local crs we define.

In [ ]:
# Local crs to use in model
crs = "EPSG:25833"

# model resolution in meters
res = 20

# waterlevel forcing point
x = [372368.1]
y = [6033653.6]

# Observation points for model validation
obs = {'Station': ["Barhoeft","Barth","Althagen"], 
     'geometry': [
         Point(371100.0, 6034300.0), 
         Point(352691.1, 6027443.5),
         Point(332296.2, 6027473.9),
         ]}



## Base SFINCS setup

### Base SFINCS object

In [ ]:
# topobathy datalib
datalib = data_root/"topobathy"/"data.yml"

logger = setuplog(log_level=20)

sf = SfincsModel(
    root=model_root/model_name,
    data_libs=datalib,
    mode="w+",
    logger=logger
)

### Initial settings & grid
some will be overwritten either later during setup (zsini) or when updating for an event (timing).\
A custom mask file is used to make sure the entire desired area is covered by the model.

In [ ]:
# mask file
mask_file = data_root/"mask"/"mask_25833.geojson"

sf.setup_config(
    # timings will be overwritten later in scenario setup
    tref="20100101 000000",
    tstart="20100101 000000",
    tend="20100102 000000",
    # misc time options
    dtout=10400.0,
    dthisout=600.0,
    dtmaxout=259200.0,
    tspinup=86400,
    # zsini will be overwritten later in base setup
    zsini=-1,
)

# Create grid
sf.setup_grid_from_region(
    region={"geom": mask_file},
    res=res,
    crs=crs, # local crs
    rotated=False
)

### Topobathy
The data is taken from the datacatalog defined when creating the `SfincsModel` object. The datasets used below are refered to by their name given in that datacatalog.

In [ ]:
# setup topobathy
datasets_dep = [{"elevtn": "MVtopo_masked", "zmin": 0.01},{"elevtn": "ELC_INSPIRE"},{"elevtn": "merit_hydro"}]
dep = sf.setup_dep(datasets_dep=datasets_dep)

### (In)active & boundary cells
Indicating which cells are (in)active or are boundary cells partly depends on the bathymetry, hence can only be done after setting up the topobathy/\
Here we also set up the friction coefficients for land and sea celss

In [ ]:
# setup active mask
sf.setup_mask_active(
    include_mask=mask_file,
    zmax=-50.0,
)
sf.setup_mask_bounds(
    btype="waterlevel",
    zmax=0.0,
)

# setup roughness
sf.setup_manning_roughness(
    manning_land=0.04,
    manning_sea=0.02,
    rgh_lev_land=0,  # the minimum elevation of the land
)

### Initial waterlevels
We use a manually created file to indicate what the initial waterlevels. The topography of the area is complicated, with multiple regions below sea level. Using a flat initial waterlevel of 0 or -1 will have these regions start already flooded. Using a low enough initial level so that these areas remain dry will require a longer spinup time to make sure the water reaches an equilibrium position for the event. For this particular area it can take a significant amount of time for the water to properly settle in all corners.

In [ ]:
# setup initial water levels
# Initial water levels file
ini_wl_file = data_root/"mask"/"zsini_regridded_20m_final.tif"

ini_ds = xr.open_dataset(ini_wl_file, engine="rasterio")
ini_ds = ini_ds.rename({"band_data": "zsini"})
sf.set_states(data=ini_ds["zsini"])
sf.set_config("inifile", "sfincs.ini")

## Setup waterlevel forcing point and obs stations
We use a dummy timeseries for the setup of the waterlevel forcing just to get the right boundary point set correctly. The time and waterlevel values themselves will be overwritten when updating the model for an event.\
We also include three observation points for comparison with observational data. These points are matched to known locations of waterlevel gauges.

In [ ]:
pnts = gpd.points_from_xy(x, y)
index = [1]  # NOTE that the index should start at one
bnd = gpd.GeoDataFrame(index=index, geometry=pnts, crs=sf.crs)

# Create some placeholder for waterlevel timeseries
# Will be overwritten in scenario setup
time = pd.date_range(
    start=utils.parse_datetime(sf.config['tstart']),
    end=utils.parse_datetime(sf.config['tend']),
    periods=3
)

bzs = [0.0, 1.0, 0.0]
bzspd = pd.DataFrame(index=time, data=bzs, columns=index)

sf.setup_waterlevel_forcing(timeseries=bzspd, locations=bnd, buffer=1000)

# Setup obs points
gdf = gpd.GeoDataFrame(obs, crs=sf.crs)
sf.setup_observation_points(
    locations=gdf, merge=False
)

## Setup dikes
Finally we configure the known dike locations in the model. The file containing the dike locations is prepared beforehand and is a merger of dike locations from several local sources of varying regional level.

In [ ]:
# dike geometry file
dike_file = data_root/"dikes"/"all_dikes.gpkg"

gdf_floodwall = sf.data_catalog.get_geodataframe(
    dike_file, geom=sf.region, crs=sf.crs
)

if (gdf_floodwall.geometry.type == "MultiLineString").any():
    gdf_floodwall = gdf_floodwall.explode()

gdf_floodwall["par1"] = 0.6

sf.setup_structures(
    structures=gdf_floodwall,
    stype="weir",
    merge=False,
)

## Save model

In [ ]:
sf.write()